In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.m2.ssro import ssro, sequence
import scipy.fftpack
import win32com.client #imports the pywin32 library

%matplotlib inline

In [ ]:
# Which data to analyse
data_disk = 'X' # For analysis from a different computer.
data_when = 'latest'
data_multiple_msmt = False
data_newer_than='20180321182000' 
data_older_than='20180322110000'

In [ ]:
def fourier(signal, time):
    """Returns the Fouier transform of a single signal with time, including the negative frequencies."""
    t_step = (time[-1] - time[0])/time.size
    fft = scipy.fftpack.fft(signal) * t_step
    freq = scipy.fftpack.fftfreq(time.size, t_step)
    return np.abs(np.fft.fftshift(fft)), np.fft.fftshift(freq)

def fourier_pos_freq(signal, time):
    """Returns the Fouier transform of a single signal with time."""
    fft, freq = fourier(signal,time)
    n = len(fft)
    fft = fft[0:n//2]
    freq = freq[0:n//2]
    return fft, freq

def folder_paths(disk='D', when='latest', multiple_msmt=False, older_than=None, newer_than=None):
    """Returns a list with string with the paths to the measurements."""
    # Find path to folder with data
    if disk == 'D':
        folder = None #'D:/measuring/data'
    else:
        folder = '{}:/data'.format(disk)
    # How to search the data
    if multiple_msmt == False:
        if when == 'latest':
            folders = tb.latest_data('mw_pulse_msmt', folder=folder)
        else:
            folders = tb.data_from_time(when, folder=folder_with_data)
    else:
        folders = tb.latest_data('mw_pulse_msmt', folder='X:/data'
                                 ,newer_than=newer_than, older_than=older_than, return_all=True)
    if type(folders) == str:
        folders = [folders]
    return folders

# def read_data2():
#     """"""
#     f = sequence.SequenceAnalysis(fol, hdf5_mode='r')
#     waveform = np.array(f.g['oscilloscope_msmt']['data'])
#     number_of_pulses = int(f.g.attrs['multiplicity'])
#     frequency = f.g.attrs['mw_frq']
    
#     return time, waveform

def read_data(folder):
    # Get the latest data from the folder.
    global t, amplitude, fft, freq, \
        number_of_measurements, number_of_pulses, points_per_pulse, \
        frequency, timestamp
    folders = folder_paths(disk=data_disk, when=data_when, multiple_msmt=data_multiple_msmt, 
                           older_than=data_older_than, newer_than=data_newer_than)
    number_of_measurements = len(folders)
    timestamp = []
    # Make numpy arrays to store the information in and make them global.
    t = np.zeros((number_of_measurements, number_of_pulses, points_per_pulse))
    amplitude = np.zeros((number_of_measurements, number_of_pulses, points_per_pulse))
    fft = np.zeros((number_of_measurements, number_of_pulses, points_per_pulse//2))
    freq = np.zeros((number_of_measurements, number_of_pulses, points_per_pulse//2))
    for msmt, fol in enumerate(folders):
#         timestamp.append(fol[18:33])
        timestamp.append(fol[8:23])
        f = sequence.SequenceAnalysis(fol, hdf5_mode='r') 
        waveform = np.array(f.g['oscilloscope_msmt']['data'])
        number_of_pulses = int(f.g.attrs['multiplicity'])
        points_per_pulse = int(1e6 // number_of_pulses)
        frequency = f.g.attrs['mw_frq']
        # Get the information of all the pulses
        for pulse in range(number_of_pulses):
            t[msmt][pulse] = waveform[0][pulse*points_per_pulse:(pulse+1)*points_per_pulse]
            amplitude[msmt][pulse] = waveform[1][pulse*points_per_pulse:(pulse+1)*points_per_pulse]    
            fft[msmt][pulse], freq[msmt][pulse] = fourier(amplitude[msmt][pulse], t[msmt][pulse])

def hermite_pulse(env_amplitude=1, mu=0, T_herm = 1.667e-8):
    t = np.linspace(-25e-8,25e-8,int(1e4)) + mu
    env = env_amplitude*(1-0.667*((t-mu)/T_herm)**2)*np.exp(-((t-mu)/T_herm)**2)
    pulse = env * np.sin(t*2*np.pi)#*frequency)
    fft, freq = fourier(pulse, t)
    return pulse, t, fft, freq
        
def plot_ideal_fft(amplitude=1.25):
    pulse, t, fft, freq = hermite_pulse(amplitude)
    plt.plot(freq, fft, 'r--', label='Ideal')
    
def set_limits():
    ind = np.argmin(np.abs(freq[0][0] - 1.72e9))
    plt.xlim(1.64e9, 1.8e9)
    plt.ylim(0, 1.2*fft[0][0, ind])

In [ ]:
read_data()
msmt = 0
plt.figure(figsize=(10,8))
for pulse in range(number_of_pulses):
    plt.plot(t[msmt][pulse], amplitude[msmt][pulse])
# plt.ylim(0,.3e-7)
plt.show()

In [ ]:
read_data()
for pulse in range(number_of_pulses):
    fft_neg, freq_neg = fourier_incl_negative(amplitude[0][pulse], t[0][pulse])
    plt.plot(freq_neg, fft_neg)
plt.title('Fourier transforms of 31 Hermite envelopes, \ntimestamp : {}'.format(timestamp[msmt]))
plt.xlabel('Frequency (Hz)')
plt.xlim(-.6e8,.6e8)
plt.show()

# Noise of oscilloscope

In [ ]:
read_data()
noise_oscilloscope = np.std(fft)
print "std of fft {:.3e}".format(noise_oscilloscope)
noise_oscilloscope = np.std(amplitude)
print "std of amplitude {:.3e}".format(noise_oscilloscope)

# Analysis of one (first) pulse

In [ ]:
read_data()
msmt = 0
pulse = 0

# Plot waveform
plt.figure(figsize=(8,8))
plt.plot(t[msmt][pulse], amplitude[msmt][pulse])
plt.xlabel('s')
plt.ylabel('V')
plt.title('Single Hermite pulse, timestamp : {}'.format(timestamp[msmt]))
deviation_from_middle = 4e-8#9e-8
middle = (t[msmt][pulse][-1]-t[msmt][pulse][0])/2 + t[msmt][pulse][0] + deviation_from_middle
plt.xlim(middle-1e-7, middle+1e-7)
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
pulse2, t2, fft2, freq2 = hermite_pulse(2.54e4,mu=2.339e-6)
plt.plot(t, pulse2-5e-3)
plt.show()

# Plot the fourier transform
plt.figure(figsize=(8,5))
plt.plot(freq[msmt][pulse], fft[msmt][pulse], label='Data')
plot_ideal_fft()
plt.title('Fourier transform, timestamp : {}'.format(timestamp[msmt]))
plt.xlabel('Frequency [Hz]')
plt.legend()
# set_limits()
plt.xlim(0,1e8)
plt.show()

# Analysis of the pulses in one sweep

In [ ]:
read_data()
msmt = 0

mean = np.mean(fft[msmt][:], axis=0)
std = np.std(fft[msmt], axis=0)
plt.figure(figsize=(10,8))
plt.plot(freq[msmt][0], mean, 'k', label='Mean')
plt.fill_between(freq[msmt][0],  mean - std, mean + std, facecolor='blue', alpha=0.5, label=r'$\pm$ 1 SD')
set_limits()
plot_ideal_fft()
plt.xlabel('f (Hz)')
plt.title('Mean Fourier transform of {} consecutive pulses , timestamp : {}'.format(number_of_pulses, timestamp[msmt]))
plt.legend()
plt.show()

plt.figure(figsize=(10,3))
plt.plot(freq[msmt][0],  std, label="Noise on RF signal")
# plt.plot([1.66e-9, 1.8e-9], [noise_oscilloscope, noise_oscilloscope], 'k--', label="Noise oscilloscope")
plt.xlim(1.64e9, 1.8e9)
plt.ylim(0,5e-10)
plt.xlabel('f (Hz)')
plt.title('Standard deviation of consequtive pulses, timestamp : {}'.format(timestamp[msmt]))
plt.show()
ind_low = int(np.argmin(np.abs(freq[msmt][0] - 1.705e9)))
ind_high = int(np.argmin(np.abs(freq[msmt][0] - 1.735e9)))
mean_plateau = np.mean(mean[ind_low:ind_high])
print "The relative mean standard deviation to the peak is {:.3e}.".format(np.mean(std)/mean_plateau)

In [ ]:
# read_data()
msmt = 3
plt.figure(figsize=(10,8))
for pulse in range(1,number_of_pulses):
    plt.plot(freq[msmt][pulse], fft[msmt][pulse], label='{}'.format(pulse))
set_limits()
# plt.ylim(0,.3e-7)

plt.xlabel('f (Hz)')
plt.title(' Fourier transforms of {} consecutive pulses , timestamp : {}'.format(number_of_pulses, timestamp[msmt]))
plt.legend()
plt.show()

In [ ]:
# read_data()
mean = np.zeros((number_of_measurements, points_per_pulse/2))
for msmt in range(number_of_measurements):
    mean[msmt] = np.mean(fft[msmt], axis=0)

plt.figure(figsize=(10,8))
plt.plot(freq[0][0], mean[0])#, label='{}'.format(msmt))
for msmt in range(number_of_measurements):
    plt.plot(freq[0][0], mean[msmt], label='{}'.format(msmt))
set_limits()
plt.title('Mean Fourier transforms of {} measurements of {} consecutive pulses, \ntimestamps : {} - {}'
          .format(number_of_measurements, number_of_pulses, timestamp[number_of_measurements-1], timestamp[0]))
plt.xlabel('f (Hz)')
plt.show()